## Importing Libraries

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr

c:\Users\Valentin\Documents\VJDS\Programming\youtube\krishNaik\AgenticCloud-Live\Feb2-Langchain\khrisLang\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading API's

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if anthropic_api_key:
    print(f"Deepseek API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Deepseek API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Deepseek API Key exists and begins sk-ant-
Google API Key not set


## Making the connections

In [3]:
openai = OpenAI()

claude = anthropic.Anthropic()

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

In [4]:
def chat():
    pass

## Gradio Interface

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():        
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():        
        model_selector = gr.Dropdown(["GPT", "Claude", "Deepseek"], label="Select model", value="GPT")
    with gr.Row():
        clear = gr.Button("Clear")        

    def do_entry(message, history, model):
        history += [{"role": "user", "content": message}]
        return "", history

    entry.submit(fn=do_entry, inputs=[entry, chatbot, model_selector], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )

    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

# ui.launch(inbrowser=True, server_port=7868,prevent_thread_lock=True)
ui.launch(inbrowser=True)


c:\Users\Valentin\Documents\VJDS\Programming\youtube\krishNaik\AgenticCloud-Live\Feb2-Langchain\khrisLang\Lib\site-packages\gradio\utils.py:1018: UserWarning: Expected 0 arguments for function <function chat at 0x0000029BE22A3D80>, received 1.
  warnings.warn(
c:\Users\Valentin\Documents\VJDS\Programming\youtube\krishNaik\AgenticCloud-Live\Feb2-Langchain\khrisLang\Lib\site-packages\gradio\utils.py:1026: UserWarning: Expected maximum 0 arguments for function <function chat at 0x0000029BE22A3D80>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
